In [1]:
#define network as list of edges
from pgmpy.models import BayesianNetwork

model = BayesianNetwork([
    ('Difficulty', 'Grade'),
    ('Industriousness', 'Grade'),
    ('Industriousness', 'SAT'),
    ('Grade', 'Letter')])

c:\Users\Razi's good laptop\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#create small helper function to define cpds easily

from pgmpy.factors.discrete import TabularCPD as t
def cpd(variableName, variableCard, valueList, evidence = None, evidenceCard = None):
    cpd = t(variable= variableName, variable_card=variableCard, 
            values=valueList, evidence= evidence, evidence_card=evidenceCard)
    return cpd

cpdDif = cpd('Difficulty', 2, [[.6],[.4]])
cpdInd = cpd('Industriousness', 2, [[.7], [.3]])
cpdSAT = cpd('SAT', 2, [[.95,.2], [.05, .8]], ['Industriousness'], [2])
cpdGrd = cpd('Grade', 3, [[0.3, 0.05, 0.9,  0.5],
                           [0.4, 0.25, 0.08, 0.3],
                           [0.3, 0.7,  0.02, 0.2]], ['Industriousness', 'Difficulty'], [2,2])
cpdLet = cpd('Letter', 2, [[0.1, 0.4, 0.99],
                           [0.9, 0.6, 0.01]], ['Grade'], [3])

In [3]:
#add cpds to model
model.add_cpds(cpdDif, cpdInd, cpdSAT, cpdGrd, cpdLet)
model.check_model()

True

In [4]:
#create inference? object? I guess
from pgmpy.sampling import BayesianModelSampling as bs
#https://pgmpy.org/approx_infer/bn_sampling.html#pgmpy.sampling.Sampling.BayesianModelSampling.forward_sample
#https://pgmpy.org/_modules/pgmpy/sampling/Sampling.html
inf = bs(model)

In [5]:
#rejection sampling
from pgmpy.factors.discrete import State
rej = inf.rejection_sample(evidence= [State('SAT', 0), State('Grade', 2)], size= 30, seed= 1337)
rej

100%|██████████| 30/30 [00:00<00:00, 546.52it/s]


,Difficulty,Grade,Industriousness,SAT,Letter
0,1,2,0,0,1
1,0,2,0,0,0
2,0,2,0,0,0
3,1,2,0,0,0
4,1,2,0,0,0
5,0,2,0,0,0
6,1,2,0,0,0
7,0,2,0,0,0
8,1,2,0,0,0
9,1,2,0,0,0


In [6]:
import pgmpy.estimators.BayesianEstimator as be 

estimator = be(model, rej)
cpd_I = estimator.estimate_cpd('Industriousness')
print(cpd_I)

+--------------------+-----+
| Industriousness(0) | 0.9 |
+--------------------+-----+
| Industriousness(1) | 0.1 |
+--------------------+-----+


In [7]:
#liklihood sampling
lik = inf.likelihood_weighted_sample(evidence= [State('SAT', 0), State('Grade', 2)], size= 30, seed= 1337)
lik

Generating for node: Letter: 100%|██████████| 5/5 [00:00<00:00, 294.33it/s]


,Difficulty,Grade,Industriousness,SAT,Letter,_weight
0,0,2,0,0,0,0.285
1,0,2,1,0,0,0.004
2,0,2,0,0,0,0.285
3,0,2,0,0,0,0.285
4,0,2,0,0,0,0.285
5,0,2,0,0,0,0.285
6,0,2,1,0,0,0.004
7,1,2,1,0,0,0.040
8,1,2,0,0,0,0.665
9,0,2,0,0,0,0.285


In [8]:
estimator = be(model, lik)
cpd_LikI = estimator.estimate_cpd('Industriousness')
print(cpd_LikI)

+--------------------+----------+
| Industriousness(0) | 0.642857 |
+--------------------+----------+
| Industriousness(1) | 0.357143 |
+--------------------+----------+
